# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [17]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

import os
data_path = os.getcwd() + '/homework/data/Part02/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
print(df.head())

   Pclass                                               Name     Sex   Age  \
0       3                            Braund, Mr. Owen Harris    male  22.0   
1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2       3                             Heikkinen, Miss. Laina  female  26.0   
3       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
4       3                           Allen, Mr. William Henry    male  35.0   

   SibSp  Parch            Ticket     Fare Cabin Embarked  
0      1      0         A/5 21171   7.2500   NaN        S  
1      1      0          PC 17599  71.2833   C85        C  
2      0      0  STON/O2. 3101282   7.9250   NaN        S  
3      1      0            113803  53.1000  C123        S  
4      0      0            373450   8.0500   NaN        S  


In [18]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
df['Embarked'] = df['Embarked'].fillna('None')
mean_df = df.groupby(['Embarked'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Embarked'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Embarked'])['Fare'].median().reset_index()
max_df = df.groupby(['Embarked'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Embarked'])
temp = pd.merge(temp, median_df, how='left', on=['Embarked'])
temp = pd.merge(temp, max_df, how='left', on=['Embarked'])
temp.columns = ['Embarked', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']
print(temp)

  Embarked  Fare_Mean  Fare_Mode  Fare_Median  Fare_Max
0        C  59.954144     7.2292        29.70  512.3292
1     None  80.000000    80.0000        80.00   80.0000
2        Q  13.276030     7.7500         7.75   90.0000
3        S  27.079812     8.0500        13.00  263.0000


In [19]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
df = pd.merge(df, temp, how='left', on=['Embarked'])
df = df.drop(['Embarked'] , axis=1)
print(df.head())

num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1) 

   Pclass                                               Name     Sex   Age  \
0       3                            Braund, Mr. Owen Harris    male  22.0   
1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2       3                             Heikkinen, Miss. Laina  female  26.0   
3       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
4       3                           Allen, Mr. William Henry    male  35.0   

   SibSp  Parch            Ticket     Fare Cabin  Fare_Mean  Fare_Mode  \
0      1      0         A/5 21171   7.2500   NaN  27.079812     8.0500   
1      1      0          PC 17599  71.2833   C85  59.954144     7.2292   
2      0      0  STON/O2. 3101282   7.9250   NaN  27.079812     8.0500   
3      1      0            113803  53.1000  C123  27.079812     8.0500   
4      0      0            373450   8.0500   NaN  27.079812     8.0500   

   Fare_Median  Fare_Max  
0         13.0  263.0000  
1         29.7  512.3292  
2    

# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [21]:
# 原始特徵 + 邏輯斯迴歸
df_minus = df.drop(['Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max'] , axis=1)

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.7038635542329971


In [23]:
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.7005180178502843
